# Operation of a Basic Ethernet Switch or Bridge
<i>Adapted for use with FABRIC from [Operation of a basic Ethernet switch or bridge](https://witestlab.poly.edu/blog/basic-ethernet-switch-operation/)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)

Switches and bridges improve performance by partitioning the collision domain, the section of network where frames will collide and destroy each other. Only one host within a collision domain may transmit, else collisions will be present. Thus, network capacity is shared with all hosts in a collision domain. Using a learning switch or bridge, each port on a switch becomes its own collision domain, so each network segment can separately support the full network capacity.

## 1. Design the Experiment
### 1.1 Reserve Resources

#### Import the Fabric API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

#### Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.

In [ ]:
slice = fablib.get_slice("BridgeOp")
slice.list_nodes()

#### Set IPs
Setup a unique IP to all nodes except the "Bridge"

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    bridge = slice.get_node(name="bridge") 
    host1 = slice.get_node(name="host1") 
    host2 = slice.get_node(name="host2")
    host3 = slice.get_node(name="host3") 
    host4 = slice.get_node(name="host4")
    
    subnet1 = IPv4Network("12.0.0.0/24")
    subnet2 = IPv4Network("13.0.0.0/24")
    subnet3 = IPv4Network("14.0.0.0/24")
    subnet4 = IPv4Network("15.0.0.0/24")
    
    host1_iface = host1.get_interface(network_name="Lan1")
    host1_iface.ip_addr_add(addr="12.0.0.1", subnet=subnet1)
    
    host2_iface = host2.get_interface(network_name="Lan2")
    host2_iface.ip_addr_add(addr="13.0.0.1", subnet=subnet2) 
    
    host3_iface = host3.get_interface(network_name="Lan3")
    host3_iface.ip_addr_add(addr="14.0.0.1", subnet=subnet3)
    
    host4_iface = host4.get_interface(network_name="Lan4")
    host4_iface.ip_addr_add(addr="15.0.0.1", subnet=subnet4) 
    
    bridge_iface1 = bridge.get_interface(network_name="Lan1")
    bridge_iface1.ip_addr_add(addr="12.0.0.2", subnet=subnet1)
    
    bridge_iface2 = bridge.get_interface(network_name="Lan2")
    bridge_iface2.ip_addr_add(addr="13.0.0.2", subnet=subnet2)
    
    bridge_iface3 = bridge.get_interface(network_name="Lan3")
    bridge_iface3.ip_addr_add(addr="14.0.0.2", subnet=subnet3)
    
    bridge_iface4 = bridge.get_interface(network_name="Lan4")
    bridge_iface4.ip_addr_add(addr="15.0.0.2", subnet=subnet4)
    bridge.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    
    for node in slice.get_nodes():
        node.execute("sudo apt install net-tools; sudo ifconfig ens7 up;")
    host1.execute("sudo ip route add 13.0.0.0/24 via 12.0.0.2;sudo ip route add 14.0.0.0/24 via 12.0.0.2;sudo ip route add 15.0.0.0/24 via 12.0.0.2;")
    host2.execute("sudo ip route add 12.0.0.0/24 via 13.0.0.2;sudo ip route add 14.0.0.0/24 via 13.0.0.2;sudo ip route add 15.0.0.0/24 via 13.0.0.2;")
    host3.execute("sudo ip route add 13.0.0.0/24 via 14.0.0.2;sudo ip route add 12.0.0.0/24 via 14.0.0.2;sudo ip route add 15.0.0.0/24 via 14.0.0.2;")
    host4.execute("sudo ip route add 13.0.0.0/24 via 15.0.0.2;sudo ip route add 14.0.0.0/24 via 15.0.0.2;sudo ip route add 12.0.0.0/24 via 15.0.0.2;")
    
except Exception as e:
    print(f"Exception: {e}")

## 2. Set up the bridge

#### Install the bridge software

In [ ]:
bridge =  slice.get_node(name="bridge") 
bridge.execute("sudo apt-get update; sudo apt-get -y install bridge-utils; sudo apt-get -y install net-tools;")

#### Flush bridge interfaces
A bridge operates at layer 2, meaning that the bridge interfaces should not have an IP address.

In [ ]:
bridge.execute("sudo ip addr flush dev ens7;\
sudo ip addr flush dev ens8;\
sudo ip addr flush dev ens9;\
sudo ip addr flush dev ens10;")

#### Create a new bridge interface `br0`

In [ ]:
bridge.execute("sudo brctl addbr br0")

#### Add the four interfaces to the bridge

In [ ]:
bridge.execute("sudo brctl addif br0 ens7;\
sudo brctl addif br0 ens8;\
sudo brctl addif br0 ens9;\
sudo brctl addif br0 ens10;")

#### Bring up all interfaces

In [ ]:
bridge.execute("sudo ifconfig ens7 up;\
sudo ifconfig ens8 up;\
sudo ifconfig ens9 up;\
sudo ifconfig ens10 up;\
sudo ifconfig br0 up;")

#### List the bridge ports

bridgeports = bridge.execute("brctl show br0")
print(bridgeports[0])

#### List the bridge known MAC addresses

In [ ]:
bridgemacs = bridge.execute("brctl showmacs br0")
print(bridgemacs[0])

### Connect to each node with an SSH terminal and ping each of the other nodes with the following commands:

`ping -c 1 12.0.0.1 ` 
    <br>`ping -c 1 13.0.0.1 ` 
    <br>`ping -c 1 14.0.0.1 ` 
    <br>`ping -c 1 15.0.0.1 `
    
Immediately after pinging, execute the following code block:

In [ ]:
bridgemacs = bridge.execute("brctl showmacs br0")
print(bridgemacs[0])